In [40]:
pip install gemact==0.1.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 45 kB 2.1 MB/s 
  Attempting uninstall: gemact
    Found existing installation: gemact 0.1.3
    Uninstalling gemact-0.1.3:
      Successfully uninstalled gemact-0.1.3


In [1]:
import gemact

In [2]:
gemact.__version__

'0.1.4'

# Different models to compute the aggregate cost

$$ \widetilde{S}=\sum_{i=1}^{\widetilde{N}} \widetilde{{Y_i}} $$

Under the following assumptions:

1. $\widetilde{S} \sim g(S)$ and $\widetilde{Y_i} \sim f(Y)$

2. Given $\widetilde{N}=n$,  the severity $\widetilde{Y_1},\ldots,\widetilde{Y_n}$ is i.i.d and does not depend on $n$.

3. $\widetilde{N}$ does not depend on $\widetilde{Y_1},\ldots,\widetilde{Y_n}$ in any way.

\end{itemize}

GEMAct requires distributive assumptions on frequency and severity. In the following tutorial we will assume:

* $\widetilde{N} \sim Poisson(4)$
* $\widetilde{Y_i} \sim Genpareto(shape=.2, scale=1, loc=0)$



In [3]:
#poisson -genpareto

fq_dist='poisson' #str
fq_par={'mu':4} #dict
sev_dist='genpareto' #str 
sev_par = {'c': .2, 'scale': 1} #dict

n_sim=int(1e+05) #int



lm_mc=gemact.LossModel(aggr_loss_dist_method='mc',
                 fq_par = fq_par,
                 fq_dist = fq_dist,
                 sev_par = sev_par,
                 sev_dist = sev_dist,
                 n_sim = n_sim,
                 random_state = 1)


INFO:lossmodel|..Approximating aggregate loss distribution via Monte Carlo simulation..
INFO:lossmodel|..Simulation completed..


In [4]:
lm_rec=gemact.LossModel(aggr_loss_dist_method='recursion',
                 fq_par = fq_par,
                 fq_dist = fq_dist,
                 sev_par = sev_par,
                 sev_dist = sev_dist,
                 n_sev_discr_nodes = int(1000),
                 sev_discr_step = 1,
                 n_aggr_dist_nodes = int(10000))

INFO:lossmodel|..Approximating aggregate loss distribution via Panjer recursion..
INFO:lossmodel|..Panjer recursion completed..


In [5]:
lm_fft=gemact.LossModel(aggr_loss_dist_method='fft',
                 fq_par = fq_par,
                 fq_dist = fq_dist,
                 sev_par = sev_par,
                 sev_dist = sev_dist,
                 n_sev_discr_nodes = int(1000),
                 sev_discr_step = 1,
                 n_aggr_dist_nodes = int(10000))

INFO:lossmodel|..Approximating aggregate loss distribution via FFT..
INFO:lossmodel|..Distribution via FFT completed..


In [6]:
print('MC',lm_mc.aggr_loss_mean())
print('RECURSIVE',lm_rec.aggr_loss_mean())
print('FFT',lm_fft.aggr_loss_mean())

MC 4.986130567647585
RECURSIVE 4.999999984652499
FFT 4.9999999846531935


# (Re)insurance contracts pricing

Below we show the following pricing models:

* Excess-of-loss (XL),
* Pricing with reinstatements,
* Stop-loss.

# Excess-of-loss (XL) pricing

In [18]:
lossXL=gemact.LossModel(aggr_loss_dist_method='fft',
             fq_par={'mu':.5},
             fq_dist= 'poisson',
             sev_par= {'loc':0,
                       'scale': 83.33333333333334, 
                       'c': 0.8333333333333334},
             sev_dist='genpareto',
             deductible=100,
             cover=100,
             sev_discr_method='massdispersal',
             n_sev_discr_nodes = int(10000),
             sev_discr_step = .01,
             n_aggr_dist_nodes = int(100000))


INFO:lossmodel|Discretization step set to cover/n_sev_discr_nodes.
INFO:lossmodel|..Approximating aggregate loss distribution via FFT..
INFO:lossmodel|..Distribution via FFT completed..


In [19]:
lossXL.pricing()

                     Contract specification            parameter                value
                               Deductible                    d                100.0
                                    Cover                u - d                100.0
                           Upper priority                    u                200.0
                     Aggregate deductible                    L                100.0
                     Quota share ceded portion                alpha                    1
                             Pure premium                    P   1.3354758296850144

 Reinstatement layer loading c:  []
fft 	 n_sev_discr_nodes m:  9999 	 n_aggr_dist_nodes n:  100000


# Pricing with reinstatements


In [9]:
lossm=gemact.LossModel(aggr_loss_dist_method='fft',
             fq_par={'mu':.5},
             fq_dist= 'poisson',
             sev_par= {'loc':0,
                       'scale': 83.33333333333334, 
                       'c': 0.8333333333333334},
             sev_dist='genpareto',
             sev_discr_method='massdispersal',
             n_sev_discr_nodes = int(10000),
             sev_discr_step = .01,
             n_aggr_dist_nodes = int(100000),
             cover=100,
             aggr_n_deductible=0,
             reinst_loading=1,
             n_reinst=1)


INFO:lossmodel|Discretization step set to cover/n_sev_discr_nodes.
INFO:lossmodel|..Approximating aggregate loss distribution via FFT..
INFO:lossmodel|..Distribution via FFT completed..


In [10]:
lossm.pricing()

                     Contract specification            parameter                value
                               Deductible                    d                  0.0
                                    Cover                u - d                100.0
                           Upper priority                    u                100.0
                     Aggregate deductible                    L                  0.0
                     Quota share ceded portion                alpha                    1
                     Number of reinstatements                    K                    1
                             Pure premium                    P    24.97739773475006

 Reinstatement layer loading c:  [1]
fft 	 n_sev_discr_nodes m:  9999 	 n_aggr_dist_nodes n:  100000


# Stop Loss pricing

In [11]:
lossm=gemact.LossModel(aggr_loss_dist_method='fft',
             fq_par={'mu':.5},
             fq_dist= 'poisson',
             sev_par={'loc':0,'scale': 83.33333333333334, 'c': 0.8333333333333334},
             sev_dist='genpareto',
             sev_discr_method='localmoments',
             n_sev_discr_nodes=int(10000),
             sev_discr_step=.01,
             n_aggr_dist_nodes=int(100000),
             K=5)

INFO:lossmodel|..Approximating aggregate loss distribution via FFT..
INFO:lossmodel|..Distribution via FFT completed..


In [12]:
lossm.pricing()

                     Contract specification            parameter                value
                               Deductible                    d                  0.0
                                    Cover                u - d                  inf
                           Upper priority                    u                  inf
                     Aggregate deductible                    L                  0.0
                     Quota share ceded portion                alpha                    1
                             Pure premium                    P    8.525107865235272

 Reinstatement layer loading c:  []
fft 	 n_sev_discr_nodes m:  10000 	 n_aggr_dist_nodes n:  100000


# Severity discretization

In order to pass from a continuous distribution to an arithmetic distribution, it is important to preserve the distribution properties either locally or globally. Given a span $h$ and $M$ the last available point, in GEMAct there are two available solutions.

**Method of mass dispersal**

$$ f_{0}=\operatorname{Pr}\left(\widetilde{Y}<\frac{h}{2}\right)=F_{\widetilde{Y}}\left(\frac{h}{2}-0\right)$$

$$ f_{j}=F_{\widetilde{Y}}\left(j h+\frac{h}{2}-0\right)-F_{\widetilde{Y}}\left(j h-\frac{h}{2}-0\right), \quad j=1,2, \ldots, M-1$$


$$f_{M}=1-F_{X}[(M-0.5) h-0]$$

**Method of local moments matching**

The following approach is applied to preserve the global mean of the distribution.

$$f_0 = m^0_0$$

$$f_j = m^{j}_0+ m^{j-1}_1 , \quad j=0,1, \ldots, M$$

$$ \sum_{j=0}^{1}\left(x_{k}+j h\right)^{r} m_{j}^{k}=\int_{x_{k}-0}^{x_{k}+ h-0} x^{r} d F_{X}(x), \quad r=0,1$$

$$ m_{j}^{k}=\int_{x_{k}-0}^{x_{k}+p h-0} \prod_{i \neq j} \frac{x-x_{k}-i h}{(j-i) h} d F_{X}(x), \quad j=0,1$$


In [13]:
import numpy as np

lossm_md=gemact.LossModel(aggr_loss_dist_method='fft',
             fq_par={'mu':.5},
             fq_dist= 'poisson',
             sev_par={'loc':0,'scale': 83, 'c': 0.83},
             sev_dist='genpareto',
             cover=100,
             sev_discr_method='massdispersal',
             n_sev_discr_nodes = int(10000),
             sev_discr_step = .01,
             n_aggr_dist_nodes = int(100000))

lossm_lm=gemact.LossModel(aggr_loss_dist_method='fft',
             fq_par={'mu':.5},
             fq_dist= 'poisson',
             sev_par={'loc':0,'scale': 83, 'c': 0.83},
             sev_dist='genpareto',
             cover=100,
             sev_discr_method='localmoments',
             n_sev_discr_nodes = int(10000),
             sev_discr_step = .01,
             n_aggr_dist_nodes = int(100000))




INFO:lossmodel|Discretization step set to cover/n_sev_discr_nodes.
INFO:lossmodel|..Approximating aggregate loss distribution via FFT..
INFO:lossmodel|..Distribution via FFT completed..
INFO:lossmodel|Discretization step set to cover/n_sev_discr_nodes.
INFO:lossmodel|..Approximating aggregate loss distribution via FFT..
INFO:lossmodel|..Distribution via FFT completed..


Below we show the computations consistency by comparing the discrete mean via mass dispersal and the discrete mean via local moments discretization.

In [14]:
massD=lossm_md.severity_discretization()
localM=lossm_lm.severity_discretization()

meanMD = np.sum(massD['sev_nodes']*massD['fj'])
meanLM = np.sum(localM['sev_nodes']*localM['fj'])

In [15]:
print('Mean (mass dispersal): ',meanMD)
print('Mean (local moments): ',meanLM)


Mean (mass dispersal):  64.61227417939517
Mean (local moments):  64.61227421887484
